<a href="https://colab.research.google.com/github/dietmarja/LLM-Elements/blob/main/fine_tuning/fine_tuning_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tuning

In [1]:
import torch
from torch import nn, optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [2]:
# Initialize the model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

In [4]:
# Prepare the data (we use a simply dummy dataset)
texts = ["Example sentence one.", "Example sentence two."]
inputs = tokenizer(
    texts, return_tensors="pt", padding=True, truncation=True, max_length=128
)


In [5]:
# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()


In [6]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    for i in range(len(inputs["input_ids"])):
        input_ids = inputs["input_ids"][i].unsqueeze(0)
        attention_mask = inputs["attention_mask"][i].unsqueeze(0)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss for every batch (we have here 1 sentence per batch)
        print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")



# Epoch: 0, Batch: 0, Loss: 7.37591552734375
# Epoch: 0, Batch: 1, Loss: 5.236732482910156
# Epoch: 1, Batch: 0, Loss: 2.4176738262176514
# Epoch: 1, Batch: 1, Loss: 2.430166006088257
# Epoch: 2, Batch: 0, Loss: 1.056653618812561
# Epoch: 2, Batch: 1, Loss: 1.1000455617904663


Epoch: 0, Batch: 0, Loss: 7.37591552734375
Epoch: 0, Batch: 1, Loss: 5.236732482910156
Epoch: 1, Batch: 0, Loss: 2.4176738262176514
Epoch: 1, Batch: 1, Loss: 2.430166006088257
Epoch: 2, Batch: 0, Loss: 1.056653618812561
Epoch: 2, Batch: 1, Loss: 1.1000455617904663
